In [13]:
# let's start by loading the environment variables

import os

from dotenv import load_dotenv

load_dotenv()

key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

assert key, "Please set the AZURE_OPENAI_API_KEY environment variable"
assert endpoint, "Please set the AZURE_OPENAI_ENDPOINT environment variable"
assert deployment_name, "Please set the DEPLOYMENT_NAME environment variable"
assert api_version, "Please set the AZURE_OPENAI_API_VERSION environment variable"

In [25]:
# Example 1: Vanilla chat experience

from openai import AzureOpenAI

client = AzureOpenAI(api_key=key, api_version=api_version, azure_endpoint=endpoint)  # type: ignore (asserted already)

response = client.chat.completions.create(
    model=deployment_name,  # type: ignore (asserted already)
    messages=[
        {
            "role": "system",
            "content": "You are an interesting know-it-all person.",
        },
        {"role": "user", "content": "Tell me one interesting fact."},
    ],
)

print(response.choices[0].message.content)

One interesting fact is that the world's oldest known recipe is for beer. It was written on a clay tablet in ancient Sumeria around 1800 BC. Cheers to our ancient beer-loving ancestors!


In [26]:
# Example 2: Introducing LangChain

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    api_version=api_version,  # type: ignore
    azure_deployment=deployment_name,
)


sys_msg = SystemMessage(content="You are an interesting know-it-all person.")
human_msg = HumanMessage(
    content="Tell me one interesting fact",
)
# introduce randomness in the response
response = model.invoke([sys_msg, human_msg], temperature=0.9, top_p=0.9, frequency_penalty=0.5, presence_penalty=0.6)

response.content

'Did you know that there is a species of jellyfish called Turritopsis dohrnii, also known as the "immortal jellyfish"? It has the remarkable ability to revert back to its juvenile form after reaching maturity, effectively restarting its life cycle. This process can occur repeatedly, potentially making it biologically immortal.'

In [12]:
# Example 3: Log Analysis

from pathlib import Path

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    api_version=api_version,  # type: ignore
    azure_deployment=deployment_name,
)

log = Path("logs.log").read_text()

sys_msg = SystemMessage(content="You are an expert software engineer.")
human_msg = HumanMessage(
    content=f"Analyze the following log file and create a short paragraph-long summary of it, highlighting anomalies:\n\n<logs>\n\n{log}\n\n</logs>",
)
# introduce randomness in the response
response = model.invoke([sys_msg, human_msg], temperature=0.3)

print(response.content)

The log file contains a series of events that occurred within a system. Some anomalies can be observed from the log. 

First, there were multiple failed login attempts for the user456, followed by a successful login. This could indicate a potential security breach or unauthorized access. Additionally, there was an unauthorized access attempt detected, resulting in the blocking of the IP address 192.168.1.100.

There were also several errors related to the payment process. A transaction failed initially but was retried and eventually succeeded. Furthermore, a fraudulent transaction was detected and flagged for review, indicating potential fraudulent activity.

Server-related anomalies include server overload, unexpected server crashes, low disk space, and multiple server failures. These issues were addressed through server restart, disk cleanup, and server failover.

Database anomalies include high memory usage, slow queries, data corruption, and high CPU usage. These issues were resolv